<a href="https://colab.research.google.com/github/GHROTH-L/crawler/blob/main/%E9%9B%BB%E5%BD%B1%E8%A9%95%E8%AB%96%E6%83%85%E7%B7%92%E5%88%86%E6%9E%90.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



1.   從ppt 電影版收集影片影評及正確標籤>取得影評標題及網址、透過網址進行前處理
2.   使用機器學習 並用svm 預測未知影評的正負評論



#先整理資料

In [127]:
import requests
import re
import csv
from bs4 import BeautifulSoup
import time

In [128]:
#收取文章的def
def get_articles(url):
  resp = requests.get(
        url=url,
        cookies={'over18': '1'}# 告知 Server 已回答過滿 18 歲的問題 
    )
  soup = BeautifulSoup(resp.text, 'html')
  prev_link = soup.find('div','btn-group btn-group-paging').find_all('a')[1] # 取第二個，因為這裡有三個連結：最舊、上一頁、最新
  prev_link = prev_link['href'] if 'href' in prev_link.attrs else None
  positive= []
  negative= []
  for div in soup.find_all('div','r-ent'):
    href = div.find('div','title').a['href'] #取得每一個連結
    title = div.find('div', 'title').text.strip()#取得每個開頭
    # 這裡要抓出正負評tag
    if re.match('\[.*\]',title):
      tag = re.match('\[.*\]',title).group(0)
      #標籤內含有 好為好評； 負 爛 則為負評
      if '好' in tag:
        positive.append([title,href])
      if '爛' in tag or '負' in tag:
        negative.append([title,href])
  return prev_link, positive, negative

In [142]:
#進入每個連結的內文，搜尋內文
#必須要捨去推文，因為推文正反意見可能大不相同
def trim(txt):
  # 保留英數字, 中文 (\u4e00-\u9fa5) 及中文標點, 部分特殊符號
  expr = re.compile('[^\u4e00-\u9fa5。；，：“”（）、？「」『』【】\s\w:/\-.()]')  #^表示"非括號內指定的字元"
  '''
\u4e00-\u9fa5 表示 Unicode 範圍，包含了所有的中文字元。
。；，：“”（）、？「」『』【】 是常見的中文標點符號。
\s 匹配任意空白字符，如空格、制表符、換行符等。
\w 匹配任意單詞字符，包括字母、數字和下劃線。
:/\-.() 匹配特定的符號，包括冒號、斜線、破折號和括號。
  '''
  txt = re.sub(expr, '', txt)#這一行使用 re.sub() 方法將 txt 字符串中匹配 expr 的部分替換為空字符串，也就是將非中文字符移除。替換後的結果存儲在 txt 變數中。
  txt = re.sub('[。；，：“”（）、？「」『』【】:/\-_.()]', ' ', txt)  # 用空白取代中英文標點
  txt = re.sub('(\s)+', ' ', txt)  # 用單一空白取代多個換行或 tab 符號
  txt = txt.replace('--', '')
  txt = txt.lower()  # 英文字轉為小寫
  return txt
  

In [173]:
#得到內文
def get_post(url):
  resp = requests.get(
    url = url,
    cookies={'over18': '1'}
  )
  soup = BeautifulSoup(resp.text, 'html')
  main_content = soup.find('div', id='main-content')
  # 把非本文的部份 (標題區及推文區) 移除
  # 移除標題區塊
  for meta in main_content.find_all('div', 'article-metaline'):
    meta.extract()                             #這在處理 HTML 或 XML 文檔時非常有用。通常，當你想從文檔中刪除特定元素或內容時，你可以使用 extract() 方法。
  for meta in main_content.find_all('div', 'article-metaline-right'):
    meta.extract()
  # 移除推文區塊
  for push in main_content.find_all('div', 'push'):
    push.extract()
  parsed = []
  for txt in main_content.stripped_strings:
      # 移除 '※ 發信站:', '--' 開頭, 及本文區最後一行文章網址部份
    if txt[0] == '※' or txt[:2] == '--' or url in txt:
      continue
    txt = trim(txt)
    if txt:
      parsed.append(txt)
  return ''.join(parsed)#parsed 是一個列表，你想要將它的元素以空字串作為分隔符進行串接。使用 ''.join(parsed) 可以將列表中的元素連接成一個字串。請注意，這只適用於列表中的元素都是字串的情況。如果列表中包含非字串元素，你需要先將其轉換為字串，然後再使用 join() 方法進行串接。

In [144]:
def get_body(ps):
  id_to_body={}
  PTT_URL = 'https://www.ptt.cc'
  for p in ps:
    title = ' '.join(p[0].split(']')[1:])
    title = trim(title)
    body = get_post(PTT_URL + p[1])
    id_to_body[p[1]] = title + ' ' + body  # 以文章超連結為 key, 標題 + 本文為 value
    time.sleep(1)  # 放慢爬蟲速度
  return id_to_body


In [183]:
#主程式 分類並取出標題 還有連結
if __name__ == "__main__":
  ptt_url = 'https://www.ptt.cc/bbs/movie/search?q='
  movie = str(input("請輸入要查找的電影名稱"))
  url = ptt_url +movie
  pp, np=[],[]
  prev_link, positive, negative =  get_articles(url)
  pp= positive + pp
  np = negative + np
  while prev_link:
    url = 'https://www.ptt.cc' + prev_link
    prev_link, positive, negative =  get_articles(url)
    pp= positive + pp
    np = negative + np
  print(len(pp), pp[:3])
  print(len(np),np[:3])
  d1 = get_body(pp)
  d2 = get_body(np)
  id_to_body = {**d1, **d2}
  


請輸入要查找的電影名稱黑豹
74 [['[ 好雷] 美國隊長3 黑豹疑問', '/bbs/movie/M.1462012717.A.D23.html'], ['[好雷]美隊3-關於黑豹以及各英雄的立場', '/bbs/movie/M.1461921066.A.E77.html'], ['[好雷] 美國隊長3  黑豹家世', '/bbs/movie/M.1461748087.A.515.html']]
56 [['[負雷]黑豹', '/bbs/movie/M.1518495315.A.E27.html'], ['[  負雷] 讓人失望的黑豹', '/bbs/movie/M.1518529744.A.586.html'], ['[普負雷]  一片歐罵罵的黑豹', '/bbs/movie/M.1518513719.A.462.html']]


In [172]:
id_to_body

{'/bbs/movie/M.1681401125.A.15A.html': ' 重看了一遍小美人魚 隔了好久好久的重溫 以前只記得王子開船把女巫撞死 中間除了歌曲外的細節全忘了 看了之後跟小時候的體驗不同 發現這部與其說是公主追愛 主軸其實是父愛吧 人魚王的角色弧線才是最首尾呼應 最完整的 為女犧牲 為女放手 成就女主 實在太偉大了qq 好在女主最後也體會到了父親對她的愛 不過人魚在海底都吃什麼啊 ',
 '/bbs/movie/M.1685278231.A.846.html': ' 小美人魚 我還沒去看 但我看過的朋友都非常讚歎女主角的演技與歌藝 明年奧斯卡最佳女主角很有可能就是她 年紀輕輕有如此唱功與渾然天成的演技 未來影視產業會因為她而顛覆一些刻板印象 ',
 '/bbs/movie/M.1685269165.A.C83.html': ' 小美人魚比龍與地下城 好看 我之前去看被電影版大肆吹捧的龍與地下城 當時這部電影好雷響到乒乓叫 結果大失所望 被電影版騙了 根本是一部超級垃圾b級片 而且特效有夠假 這次去看電影版負評轟炸的小美人魚的黑人版 其實根本沒有想像中的差 海底世界的特效呈現就很滿足 美輪美奐 女主角的歌聲真的好聽 整體上劇情也沒什麼好批評幾乎還原度很高 我是覺得動畫中的小丑魚和螃蟹變成真實版有點恐怖 原本動畫中圓滾滾的造型變成扁平魚 反而不可愛了 結論 電影版反指標 ',
 '/bbs/movie/M.1685264714.A.1CD.html': ' 小美人魚 前面無趣 後面有意思 防雷頁開始 這部 算是我第一部小美人魚電影 先前沒看過動畫版 觀影前對於小美人魚劇情的認知幾乎為零 只看過板上幾篇文 和聽過一些歌 先上人權 很幸運選到這區影廳 好像是最近才剛蓋好 整個很乾淨 很新https imgur com a oxfvohf三排其實是中間區域 一開始還以為會很前面 良心打碼 https imgur com a h88mfu8防雷頁結束 電影開場真的讓人沒什麼記憶點 引用人魚的名言 然後抓人魚 這邊唯一不錯的 大概只有海底運鏡讓人感覺舒服 海王開會叫螃蟹去找艾莉兒 魚和螃蟹第一次說話真的讓我覺得不舒服 就是走寫實風 但發聲方式又不寫實 會讓我好奇到底怎麼發聲 星際異攻隊那隻狗至少戴著發聲項圈說話 還不會讓人覺得那麼怪 緊接就是艾莉兒第一次唱歌 

#接下來要使用機器學習

In [175]:
import jieba #判斷中文斷詞的套件 綽號 結巴
import random #隨機
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import accuracy_score

[['[好雷] 重看了一遍小美人魚', '/bbs/movie/M.1681401125.A.15A.html'],
 ['[驚世好雷] 小美人魚', '/bbs/movie/M.1685278231.A.846.html'],
 ['[偏好雷] 小美人魚比龍與地下城 好看', '/bbs/movie/M.1685269165.A.C83.html'],
 ['[普好雷] 小美人魚，前面無趣，後面有意思', '/bbs/movie/M.1685264714.A.1CD.html'],
 ['[好雷] 小美人魚 其實還不錯呀', '/bbs/movie/M.1685200494.A.FCF.html'],
 ['[好雷] 小美人魚 Die kleine Meerjungfrau (2013 德國 ARD)',
  '/bbs/movie/M.1685394627.A.9F4.html'],
 ['[好雷] 小美人魚其實很好看 沒看就罵的都是歧視', '/bbs/movie/M.1685456421.A.14F.html'],
 ['[好雷] 小美人魚/海的女兒 Rusalochka (1976 蘇聯＆保加利亞)',
  '/bbs/movie/M.1685650562.A.A5C.html'],
 ['[好雷] 小美人魚/海的女兒 Malá Morská Víla (1976捷克斯洛伐克)',
  '/bbs/movie/M.1685566374.A.CCB.html'],
 ['[好雷] 小美人魚，烏蘇拉比小美人魚精彩～', '/bbs/movie/M.1685808842.A.60A.html'],
 ['[好雷] 小美人魚 曠古空前的驚艷之作,值得滿分！', '/bbs/movie/M.1685810462.A.A8D.html']]

In [185]:
#創造依據正負面 標題以及json資料相關的連結 使用網址當作Id 將 pp/np 與json 資料連結 並且增加label
def load_data(ps,id_to_body,label): #pp/np id_to_body, 正評/負評
  ids= []
  for p in ps:
    ids.append(p[1])
  data = []
  for id in ids:
    tokenized_post=[]
    txt = id_to_body[id]
    for sent in txt.split(): #將文章用空格分開
      #斷詞後，若非空白長度2以上，加入詞庫
      filtered = [t for t in jieba.cut(sent) if t.split() and len(t)>1]
      tokenized_post = tokenized_post +filtered
    data.append((tokenized_post,label))
  return data

In [186]:
pos_data = load_data(pp,id_to_body,'正評')
neg_data = load_data(np,id_to_body,'負評')

#印出評論前幾個字確認資料
for post, label in pos_data[:3]:
  print(post[:5], label)
for post, label in neg_data[:3]:
  print(post[:5], label)

['美國隊', '黑豹', '疑問', '黑豹', '巴奇'] 正評
['美隊', '關於', '黑豹', '以及', '英雄'] 正評
['美國隊', '黑豹', '家世', '美國隊', '看來'] 正評
['黑豹', '黑豹', '覺得', '超級', '英雄'] 負評
['失望', '黑豹', '今天', '第一天', '就衝'] 負評
['一片', '歐罵', '黑豹', '今天', '黑豹'] 負評


In [188]:
print(len(neg_data))
print(len(pos_data))

56
74


#打亂資料

In [192]:
random.seed(69)
random.shuffle(pos_data)
random.shuffle(neg_data)

x_train, y_train, x_test, y_test=[], [], [], []

#將前面20的資料放入training set
#建立資料時要用塊白將對好的詞重新連成字串
#以便之後使用scikit-learn 建立字典並轉換文字為向量
for i in range(40):
  x_train.append(' '.join(pos_data[i][0]))
  x_train.append(' '.join(neg_data[i][0]))
  y_train.append(pos_data[i][1])
  y_train.append(neg_data[i][1])
#最後放入test  #neg_data比較小
for i in range(40,len(neg_data)):
  x_test.append(' '.join(pos_data[i][0]))
  x_test.append(' '.join(neg_data[i][0]))
  y_test.append(pos_data[i][1])
  y_test.append(neg_data[i][1])

In [193]:
vectorizer = CountVectorizer()
x_train = vectorizer.fit_transform(x_train)
transformer = TfidfTransformer()
x_train = transformer.fit_transform(x_train)
clf = SGDClassifier(random_state=42)
clf.fit(x_train, y_train)
x_test = vectorizer.transform(x_test)
x_test = transformer.transform(x_test)
y_pred = clf.predict(x_test)
print('預測結果:', list(y_pred))
print('正確答案:', y_test)
print('正確率:', accuracy_score(y_test, y_pred))

# 測試自己輸入的句子
sentences = [
    '我 覺得 這部 電影 還 不錯',
    '這部 片 應該 可以 更好 才對'
]
analyze = vectorizer.build_analyzer()
print(analyze(sentences[0]))
print(analyze(sentences[1]))

custom_data = transformer.transform(vectorizer.transform(sentences))
print(clf.predict(custom_data))

預測結果: ['正評', '負評', '負評', '正評', '負評', '負評', '正評', '正評', '負評', '負評', '正評', '負評', '正評', '負評', '正評', '負評', '正評', '負評', '負評', '負評', '負評', '負評', '正評', '正評', '正評', '正評', '負評', '負評', '負評', '正評', '正評', '負評']
正確答案: ['正評', '負評', '正評', '負評', '正評', '負評', '正評', '負評', '正評', '負評', '正評', '負評', '正評', '負評', '正評', '負評', '正評', '負評', '正評', '負評', '正評', '負評', '正評', '負評', '正評', '負評', '正評', '負評', '正評', '負評', '正評', '負評']
正確率: 0.625
['覺得', '這部', '電影', '不錯']
['這部', '應該', '可以', '更好', '才對']
['正評' '正評']


#group(0) 使用
在正則表達式中，group(0) 是用來取得完整匹配的子字串的方法。當使用正則表達式進行匹配時，如果找到了符合模式的子字串，可以使用 group(0) 方法來獲取這個完整的匹配結果。


In [9]:
import re

pattern = r'\b\w+\b'  # 匹配單詞的正則表達式模式
text = "Hello, World!"
match = re.search(pattern, text)

if match:
    word = match.group(0)
    print(word)  # 輸出: Hello

Hello


#re.compile 使用

In [113]:
import re

pattern = re.compile(r'\d{2}-\d{2}-\d{4}')
text = 'Today is 20-05-2023.'

result = pattern.search(text)
if result:
    print('找到匹配：', result.group())
else:
    print('未找到匹配。')

找到匹配： 20-05-2023


#re.sub 的使用
替換

In [114]:
import re

text = 'Hello, World!'
result = re.sub(r'World', 'Python', text)
print(result)

Hello, Python!
